In [1]:
from datetime import datetime
from heapq import nlargest
from operator import itemgetter
from collections import defaultdict
from sys import stdout
print('Preparing arrays...')

Preparing arrays...


In [2]:
###### finding most popular 50
f = open("train.csv", "r")
f.readline()
most_hotels_cntr_hm_sdi = defaultdict(int)
total = 0

In [3]:
import time

start = time.time()

while 1:
    line = f.readline().strip()
    total += 1

    if total % 10000000 == 0:
        print('Read {} lines...'.format(total))

    if line == '':
        break

    arr = line.split(",")
    srch_destination_id = arr[16]
    hotel_country = arr[21]
    hotel_market = arr[22]

    if srch_destination_id != '' and hotel_country != '' and hotel_market != '':
        most_hotels_cntr_hm_sdi[(hotel_country,hotel_market,srch_destination_id)] += 1

f.close()
end = time.time()
print(end - start)

Read 10000000 lines...
Read 20000000 lines...
Read 30000000 lines...
119.785424232


In [4]:
topitems = nlargest(50, most_hotels_cntr_hm_sdi.items(), key=itemgetter(1))

In [5]:
topitems_dict = defaultdict(int)

In [6]:
for k,v in topitems:
    topitems_dict[k] = v

In [9]:
############ Features transformation & Construction #############
f = open("train.csv", "r")
f.readline()
feature1_cntr_hm_sdi = defaultdict(lambda: defaultdict(list)) # feature_contstrunction
total = 0
missline = 0

In [10]:
import time

start = time.time()

while 1:
    line = f.readline().strip()
    total += 1

    if total % 10000000 == 0:
        print('Read {} lines...'.format(total))

    if line == '':
        break
    #if total > 1000: break
    
    arr = line.split(",")
    srch_destination_id = arr[16]
    hotel_country = arr[21]
    hotel_market = arr[22]

    if srch_destination_id != '' and hotel_country != '' and hotel_market != '' and ((hotel_country,hotel_market,srch_destination_id) in topitems_dict):
        try:
            search_time_DT = datetime.strptime(arr[0], '%Y-%m-%d %H:%M:%S')
            arr[0] = search_time_DT.hour      #srch_time_hour
            arr[1] = search_time_DT.weekday() #srch_time_weekday
            arr[20] = search_time_DT.year     #srch_time_year
            #arr[2] = int(arr[2])             #posa_continent
            
            arr[6] = float(arr[6]) #orig_dest_distance
        
            #arr[8] = int(arr[8]) #is_mobile
            #arr[9] = int(arr[9]) #is_package
            #arr[10] = int(arr[10]) #channel
            
            arr[21] = int(arr[11].split('-')[1])
            arr[22] = int(arr[12].split('-')[1])
            checkInDateDT = datetime.strptime(arr[11], "%Y-%m-%d")
            checkOutDateDT = datetime.strptime(arr[12], "%Y-%m-%d") 
            arr[11] = checkInDateDT.timetuple().tm_yday + 1  #srch_ci_day
            arr[12] = checkOutDateDT.timetuple().tm_yday+ 1 #srch_co_day
            
            arr[13] = int(arr[13]) #srch_adult_count
            arr[14] = int(arr[14]) #srch_child_count
            arr[15] = int(arr[15]) #srch_room_count
            
            arr[16] = arr[12] - arr[11] #duration
            #arr[17] = int(arr[17]) #srch_dest_type_id
            arr[18] = int(arr[18])  #is_booking should be weighted
            arr[19] = int(arr[19])  #hotel_cluster
        except:
            missline += 1
            continue
        colNames = ['hour','dow','posa','','','','dist','','mobile','package','channel','ci_day','co_day',
                    'adults','children','room','stay','type','booking','cluster','year','ci_month','co_month']
        colindex = [0,1,2,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]
        for i in colindex:
            feature1_cntr_hm_sdi[(hotel_country,hotel_market,srch_destination_id)][colNames[i]].append(arr[i])

f.close()
end = time.time()
print(end - start)

Read 10000000 lines...
Read 20000000 lines...
Read 30000000 lines...
628.067980051


In [24]:
feature1_cntr_hm_sdi.items()[0][1].keys()

['ci_day',
 'dist',
 'adults',
 'hour',
 'package',
 'mobile',
 'ci_month',
 'co_day',
 'children',
 'posa',
 'cluster',
 'co_month',
 'booking',
 'year',
 'stay',
 'type',
 'dow',
 'channel',
 'room']

In [11]:
########## check -in & -out info. processing   ##########
import collections as cl
check_InOut_time = defaultdict(lambda: defaultdict(list))

In [25]:
import time

start = time.time()
length = len(feature1_cntr_hm_sdi.items())
for i in range(length):
    country_market_dest = feature1_cntr_hm_sdi.items()[i][0]
    check_InOut_time[country_market_dest]['ci_month'] = [k for k in feature1_cntr_hm_sdi.items()[i][1]['ci_month']]
    check_InOut_time[country_market_dest]['co_month'] = [k for k in feature1_cntr_hm_sdi.items()[i][1]['co_month']]
    check_InOut_time[country_market_dest]['ci_day'] = [k for k in feature1_cntr_hm_sdi.items()[i][1]['ci_day']]
    check_InOut_time[country_market_dest]['co_day'] = [k for k in feature1_cntr_hm_sdi.items()[i][1]['co_day']]
    temp_ci = check_InOut_time[country_market_dest]['ci_month']
    temp_co = check_InOut_time[country_market_dest]['co_month']
    counter_ci = cl.Counter(temp_ci)
    counter_co = cl.Counter(temp_co)
    temp_ci_day = check_InOut_time[country_market_dest]['ci_day']
    temp_co_day = check_InOut_time[country_market_dest]['co_day']
    counter_ci_day = cl.Counter(temp_ci_day)
    counter_co_day = cl.Counter(temp_co_day)
end = time.time()
print (end - start)

12.5446619987


In [26]:
counter_ci_day

Counter({2: 236,
         3: 280,
         4: 124,
         5: 91,
         6: 99,
         7: 99,
         8: 46,
         9: 61,
         10: 90,
         11: 79,
         12: 43,
         13: 62,
         14: 39,
         15: 180,
         16: 62,
         17: 89,
         18: 72,
         19: 116,
         20: 40,
         21: 23,
         22: 37,
         23: 56,
         24: 100,
         25: 54,
         26: 43,
         27: 23,
         28: 62,
         29: 67,
         30: 39,
         31: 28,
         32: 69,
         33: 36,
         34: 20,
         35: 19,
         36: 41,
         37: 20,
         38: 51,
         39: 104,
         40: 38,
         41: 61,
         42: 53,
         43: 41,
         44: 144,
         45: 164,
         46: 153,
         47: 85,
         48: 54,
         49: 56,
         50: 62,
         51: 65,
         52: 77,
         53: 36,
         54: 72,
         55: 33,
         56: 86,
         57: 21,
         58: 58,
         59: 77,
         60:

In [17]:
from scipy import interpolate
import numpy as np

In [ ]:
i = 0
temp_ci = check_InOut_time.items()[i][1]['ci_month']
temp_co = check_InOut_time.items()[i][1]['co_month']
counter_ci = cl.Counter(temp_ci)
counter_co = cl.Counter(temp_co)
cix = [0]
ciy =[0]
monthday = [31,29,31,30,31,30,31,31,30,31,30,31]
total = 0 
for i in counter_ci.keys():
    total += monthday[i-1]
    cix.append(total)
total =0
for i in counter_ci.values():
    total += i
    ciy.append(total)
import matplotlib.pyplot as plt
tck = interpolate.splrep(cix, ciy, s=0)
cixnew = np.array(range(1,367))
ciynew = interpolate.splev(cixnew, tck, der=0)
#plt.plot(cix,ciy,'x',cixnew,ciynew,'b')
#plt.show()
ciyder = interpolate.splev(cixnew, tck, der=1)
x = counter_ci_day.keys()
y = counter_ci_day.values()
plt.plot(cixnew, ciyder,'--',x,y,'x')
plt.title('Derivative estimation from spline')
plt.show()

In [44]:
max(feature1_cntr_hm_sdi.items()[0][1]['co_day'])

366

In [20]:
############ check -in & -out info. processing   ############
import time
start = time.time()

length = len(feature1_cntr_hm_sdi.items())
for i in range(length):
    temp_ci = feature1_cntr_hm_sdi.items()[i][1]['ci']
    temp_co = feature1_cntr_hm_sdi.items()[i][1]['co']
    counter_ci = cl.Counter(temp_ci)
    counter_co = cl.Counter(temp_co)
    country_market_dest = feature1_cntr_hm_sdi.items()[i][0] 
    check_InOut_time[country_market_dest]['ci_day'] = [counter_ci[k] for k in temp_ci]
    feature1_cntr_hm_sdi.items()[i][1]['co'] = [counter_co[k] for k in temp_co]
end = time.time()
print(end - start)

['ci_day',
 'dist',
 'adults',
 'hour',
 'package',
 'mobile',
 'ci_month',
 'co_day',
 'children',
 'posa',
 'cluster',
 'co_month',
 'booking',
 'year',
 'stay',
 'type',
 'dow',
 'channel',
 'room']

In [9]:
check_InOut_time = defaultdict(lambda: defaultdict(list))

In [27]:
import time

start = time.time()
length = len(feature1_cntr_hm_sdi.items())
for i range(length):
    country_market_dest = feature1_cntr_hm_sdi.items()[i][0] 
    check_InOut_time[country_market_dest]['ci_month'] = feature1_cntr_hm_sdi.items()[i][1]['ci_month']
    check_InOut_time[country_market_dest]['co_month'] = feature1_cntr_hm_sdi.items()[i][1]['co_month']


Read 10000000 lines...
Read 20000000 lines...
Read 30000000 lines...
621.116348028


In [10]:
feature1_cntr_hm_sdi.items()[1][1]['ci_month']

[8,
 8,
 2,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 6,
 2,
 2,
 2,
 6,
 6,
 6,
 6,
 6,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 6,
 6,
 6,
 1,
 2,
 5,
 2,
 10,
 10,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 10,
 10,
 10,
 10,
 10,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 12,
 7,
 3,
 8,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 1,
 1,
 1,
 1,
 6,
 8,
 8,
 9,
 10,
 9,
 2,
 2,
 3,
 3,
 7,
 7,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 12,
 2,
 2,
 2,
 2,
 7,
 7,
 7,
 7,
 7,
 6,
 6,
 5,
 1,
 1,
 1,
 1,
 5,
 5,
 5,
 10,
 10,
 2,
 2,
 2,
 2,
 5,
 5,
 4,
 5,
 2,
 2,
 2,
 2,
 4,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 4,
 4,
 4,
 4,
 4,
 4,
 9,
 9,
 9,
 12,
 12,
 12,
 

In [29]:
import collections as cl

In [30]:
import time
start = time.time()

length = len(feature1_cntr_hm_sdi.items())
for i in range(length):
    temp_ci = feature1_cntr_hm_sdi.items()[i][1]['ci']
    temp_co = feature1_cntr_hm_sdi.items()[i][1]['co']
    counter_ci = cl.Counter(temp_ci)
    counter_co = cl.Counter(temp_co)
    feature1_cntr_hm_sdi.items()[i][1]['ci'] = [counter_ci[k] for k in temp_ci]
    feature1_cntr_hm_sdi.items()[i][1]['co'] = [counter_co[k] for k in temp_co]
end = time.time()
print(end - start)

9.86345601082


In [31]:
from sklearn.neighbors import KernelDensity

In [40]:
############ Features transformation & Construction #############
f = open("train.csv", "r")
f.readline()
feature2_cntr_hm_sdi = defaultdict(lambda: defaultdict(list)) # feature_contstrunction
total = 0
missline = 0

Read 10000000 lines...
Read 20000000 lines...
Read 30000000 lines...
579.584237099


In [ ]:
colNames

In [37]:
colNames = ['hour','dow','posa','','','','dist','','mobile','package','channel','ci_day','co_day',
                    'adults','children','room','stay','type','booking','cluster','year','ci_month','co_month']
len(colNames)

23

In [24]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
rfc = RandomForestClassifier(n_estimators=100)